## DuckDB Spatial



### このノートブックの流れ

- 生成 AI に、自然言語から SQL を出力させる
- 出力された SQL を DuckDB で実行する
- 実行結果を GeoJSON に変換する
- GeoJSON を地図上に表示する

### 前準備

### DuckDB のセットアップとデータの読み込み

In [1]:
import duckdb

# DuckDBに接続（インメモリで動作）
conn = duckdb.connect()

# DuckDBに拡張機能をインストール
conn.execute("""
INSTALL httpfs;
INSTALL json;
INSTALL spatial;
""")
conn.execute(f"""
LOAD httpfs;
LOAD json;
LOAD spatial;
""")
# NOTE: closeするとDBの内容が失われるのでcloseしてはいけない
# conn.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [2]:

# NOTE: connはgeojsonを読み込んだものを使い回す必要がある
# conn = duckdb.connect()

# GeoJSONファイルのURL
# admin_geojson_url = "https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_10m_admin_0_countries.geojson"
admin_geojson_url = "https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_110m_admin_0_countries.geojson"

# GeoJSON を DuckDBにテーブルとして読み込む
conn.execute(f"CREATE TABLE countries AS SELECT * FROM ST_Read('{admin_geojson_url}')")

# NOTE: closeするとDBの内容が失われるのでcloseしてはいけない
# conn.close()

### データベースのテーブルスキーマを文字列にする

In [3]:
# NOTE: connはgeojsonを読み込んだものを使い回す必要がある
# conn = duckdb.connect()
summary_of_tables = ""

# SHOWによってテーブル一覧を取得
show_result = conn.execute("SHOW").fetchall()
tables = [row[2] for row in show_result]

for table in tables:
    summary_of_tables += f"Table: {table}\n"
    # DESCRIBE TABLEの結果を文字列に変換
    describe_result = conn.execute(f"DESCRIBE TABLE {table}").fetchall()
    for row in describe_result:
        field_name = row[0]
        field_type = row[1]
        summary_of_tables += f"  Field: {field_name}, {field_type}\n"
print(summary_of_tables)

Table: countries
  Field: featurecla, VARCHAR
  Field: scalerank, INTEGER
  Field: LABELRANK, INTEGER
  Field: SOVEREIGNT, VARCHAR
  Field: SOV_A3, VARCHAR
  Field: ADM0_DIF, INTEGER
  Field: LEVEL, INTEGER
  Field: TYPE, VARCHAR
  Field: TLC, VARCHAR
  Field: ADMIN, VARCHAR
  Field: ADM0_A3, VARCHAR
  Field: GEOU_DIF, INTEGER
  Field: GEOUNIT, VARCHAR
  Field: GU_A3, VARCHAR
  Field: SU_DIF, INTEGER
  Field: SUBUNIT, VARCHAR
  Field: SU_A3, VARCHAR
  Field: BRK_DIFF, INTEGER
  Field: NAME, VARCHAR
  Field: NAME_LONG, VARCHAR
  Field: BRK_A3, VARCHAR
  Field: BRK_NAME, VARCHAR
  Field: BRK_GROUP, VARCHAR
  Field: ABBREV, VARCHAR
  Field: POSTAL, VARCHAR
  Field: FORMAL_EN, VARCHAR
  Field: FORMAL_FR, VARCHAR
  Field: NAME_CIAWF, VARCHAR
  Field: NOTE_ADM0, VARCHAR
  Field: NOTE_BRK, VARCHAR
  Field: NAME_SORT, VARCHAR
  Field: NAME_ALT, VARCHAR
  Field: MAPCOLOR7, INTEGER
  Field: MAPCOLOR8, INTEGER
  Field: MAPCOLOR9, INTEGER
  Field: MAPCOLOR13, INTEGER
  Field: POP_EST, DOUBLE
  Fie

### 対象となる自然言語

In [4]:
input_text = "日本よりも人口密度が高い国は？"
print(input_text)

日本よりも人口密度が高い国は？


### 生成 AI にスキーマと自然言語から SQL を出力させる

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# モデルの準備
model = ChatGoogleGenerativeAI(model="gemini-exp-1206", temperature=0)

# プロンプトの準備
template = """You are an expert of PostgreSQL and PostGIS.
You output the best PostgreSQL query based on given table schema and input text.

You will always reply according to the following rules:
- Output valid PostgreSQL query.
- The query MUST be return name, value and geom columns. Use AS to rename columns.
- The query MUST use ST_AsGeoJSON function to output geom column.
- The query MUST be line delimited and surrounded by just three backquote to indicate that it is a code block.
- The Value column should be a value that takes into account the user's intent to the greatest extent possible.

** Table Schema: **
{table_schema}

User Input:
{input}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

res = chain.invoke({"input": input_text, "table_schema": summary_of_tables})
result = res.content.strip()
print(result)

```sql
SELECT
  NAME AS name,
  POP_EST / ST_Area(geom) AS value,
  ST_AsGeoJSON(geom) AS geom
FROM
  countries
WHERE
  POP_EST / ST_Area(geom) > (
    SELECT
      POP_EST / ST_Area(geom)
    FROM
      countries
    WHERE
      NAME = 'Japan'
  );
```


In [6]:
import re
match = re.search(r"```[^\n]*\n(.*?)```", result, re.DOTALL)

if match:
    sql_query = match.group(1).strip()
    print(sql_query)
else:
    print("SQLが見つかりませんでした。")

SELECT
  NAME AS name,
  POP_EST / ST_Area(geom) AS value,
  ST_AsGeoJSON(geom) AS geom
FROM
  countries
WHERE
  POP_EST / ST_Area(geom) > (
    SELECT
      POP_EST / ST_Area(geom)
    FROM
      countries
    WHERE
      NAME = 'Japan'
  );


### 出力された SQL を DuckDB で実行する

In [7]:
duckdb_result = conn.execute(sql_query).fetchall()
duckdb_result

[('Haiti',
  4602594.646601417,
  '{"type":"Polygon","coordinates":[[[-71.712361,19.714456],[-71.624873,19.169838],[-71.701303,18.785417],[-71.945112,18.6169],[-71.687738,18.31666],[-71.708305,18.044997],[-72.372476,18.214961],[-72.844411,18.145611],[-73.454555,18.217906],[-73.922433,18.030993],[-74.458034,18.34255],[-74.369925,18.664908],[-73.449542,18.526053],[-72.694937,18.445799],[-72.334882,18.668422],[-72.79165,19.101625],[-72.784105,19.483591],[-73.415022,19.639551],[-73.189791,19.915684],[-72.579673,19.871501],[-71.712361,19.714456]]]}'),
 ('El Salvador',
  3696006.632444857,
  '{"type":"Polygon","coordinates":[[[-89.353326,14.424133],[-89.058512,14.340029],[-88.843073,14.140507],[-88.541231,13.980155],[-88.503998,13.845486],[-88.065343,13.964626],[-87.859515,13.893312],[-87.723503,13.78505],[-87.793111,13.38448],[-87.904112,13.149017],[-88.483302,13.163951],[-88.843228,13.259734],[-89.256743,13.458533],[-89.812394,13.520622],[-90.095555,13.735338],[-90.064678,13.88197],[-89.72

### SQL の実行結果を GeoJSON に変換する

In [8]:
# duckdb_resultをGeoJSONに変換する

import json

def duckdb_result_to_geojson(duckdb_result):
    """Converts DuckDB query result to GeoJSON format.

    Args:
        duckdb_result: The result of a DuckDB query.  Assumes the query returns
        columns named 'name', 'value', and 'geom' where 'geom' is a GeoJSON
        geometry string.

    Returns:
        A GeoJSON FeatureCollection as a string, or None if the conversion fails.
    """
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for row in duckdb_result:
        try:
            name, value, geom_str = row  # Assuming the columns are in this order
            geom = json.loads(geom_str)  # Parse the geometry string as GeoJSON
            feature = {
                "type": "Feature",
                "geometry": geom,
                "properties": {
                    "name": name,
                    "value": value
                }
            }
            geojson["features"].append(feature)
        except (ValueError, TypeError, IndexError) as e:
            print(f"Error processing row {row}: {e}")
            return None  # Return None if any row fails
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return None

    return json.dumps(geojson, indent=2)


# Example usage with the duckdb_result
geojson_output = duckdb_result_to_geojson(duckdb_result)

if geojson_output:
    print(geojson_output)
else:
    print("Failed to convert DuckDB result to GeoJSON.")

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -71.712361,
              19.714456
            ],
            [
              -71.624873,
              19.169838
            ],
            [
              -71.701303,
              18.785417
            ],
            [
              -71.945112,
              18.6169
            ],
            [
              -71.687738,
              18.31666
            ],
            [
              -71.708305,
              18.044997
            ],
            [
              -72.372476,
              18.214961
            ],
            [
              -72.844411,
              18.145611
            ],
            [
              -73.454555,
              18.217906
            ],
            [
              -73.922433,
              18.030993
            ],
            [
              -74.458034,
         

### GeoJSON を地図上に表示する

In [9]:
import folium
import json

geojson_data = json.loads(geojson_output)

m = folium.Map(location=[0, 0], zoom_start=2)  # default map

# Add the GeoJSON data to the map
folium.GeoJson(geojson_data).add_to(m)

# Get bounds from GeoJSON data and fit the map to those bounds
# Extract bounds from geojson_data
bounds = folium.GeoJson(geojson_data).get_bounds()
# Ensure bounds are valid before fitting
if bounds:
    m.fit_bounds(bounds)

# Display the map
display(m)
